In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`

import $ivy.$                                  

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder().master("local[1]").getOrCreate()

import spark.implicits._
import org.apache.spark.sql.functions._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql.SparkSession


spark: SparkSession = org.apache.spark.sql.SparkSession@7f314a04
import spark.implicits._

import org.apache.spark.sql.functions._

In [3]:
val exemplo = spark.read.textFile("data/access_log_Aug95").union(spark.read.textFile("data/access_log_Aug95"))

exemplo: org.apache.spark.sql.Dataset[String] = [value: string]

In [4]:
val splitHttpLine = udf((line: String) => {
    def bytesToLong(s: String) = if(s=="-") "0" else s
    def protocolFilter = line.replace(" HTTP/1.0", "")
    val pattern = "(.+) - - \\[(.+)\\] \"(.*)\" ([0-9]+) (-|[0-9]+)".r
    val pattern(host, date, path, code, bytes) = protocolFilter
    Array(host, date, path, code, bytesToLong(bytes))
})
val isoFormat = udf((date: String) => {
    import java.text.SimpleDateFormat
    def dateFix = date.replace("Aug", "Ago")
    val dateParser = new SimpleDateFormat("dd/MMM/yyyy:HH:mm:ss Z")
    val iso = new SimpleDateFormat("yyyy-MM-dd")
    iso.format(dateParser.parse(dateFix))
})

splitHttpLine: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  ArrayType(StringType, true),
  Some(List(StringType))
)
isoFormat: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)

In [5]:
val reqSplitted = exemplo.select(splitHttpLine(col("value")).as("req"))

reqSplitted: org.apache.spark.sql.package.DataFrame = [req: array<string>]

In [6]:
val reqStruct = reqSplitted.select(col("req")(0).as("host"), col("req")(1).as("date"), col("req")(2).as("path"), col("req")(3).as("code"), col("req")(4).as("bytes"))

reqStruct: org.apache.spark.sql.package.DataFrame = [host: string, date: string ... 3 more fields]

In [7]:
val uniqueHostsCount = reqStruct.groupBy(col("host")).count.collect.map(row=>row.getAs[String]("host")).length

uniqueHostsCount: Int = 75060

In [8]:
val codes404Count = reqStruct.where(col("code")===404).count

codes404Count: Long = 20112L

In [9]:
val top5Hosts404 = reqStruct.where(col("code")===404).groupBy("host").count.sort(col("count").desc).collect.take(5).map(row=>row.getAs[String]("host"))

top5Hosts404: Array[String] = Array(
  "dialip-217.den.mmc.com",
  "piweba3y.prodigy.com",
  "155.148.25.4",
  "maz3.maz.net",
  "gate.barr.com"
)

In [10]:
val amount404PerDay = reqStruct.where(col("code")===404).select(isoFormat(col("date")).as("dateIso")).groupBy("dateIso").count.collect.map(row=>(row.getAs[String]("dateIso"), row.getAs[Long]("count")))

amount404PerDay: Array[(String, Long)] = Array(
  ("1995-08-08", 792L),
  ("1995-08-31", 1102L),
  ("1995-08-01", 486L),
  ("1995-08-18", 514L),
  ("1995-08-21", 620L),
  ("1995-08-16", 506L),
  ("1995-08-11", 544L),
  ("1995-08-17", 552L),
  ("1995-08-26", 716L),
  ("1995-08-24", 832L),
  ("1995-08-07", 1048L),
  ("1995-08-23", 718L),
  ("1995-08-22", 532L),
  ("1995-08-25", 838L),
  ("1995-08-09", 568L),
  ("1995-08-04", 698L),
  ("1995-08-30", 1078L),
  ("1995-08-20", 612L),
  ("1995-08-13", 440L),
  ("1995-08-29", 832L),
  ("1995-08-06", 758L),
  ("1995-08-15", 648L),
  ("1995-08-03", 576L),
  ("1995-08-10", 624L),
  ("1995-08-14", 594L),
  ("1995-08-19", 426L),
  ("1995-09-01", 52L),
  ("1995-08-05", 474L),
  ("1995-08-27", 768L),
  ("1995-08-28", 812L),
  ("1995-08-12", 352L)
)

In [11]:
val totalBytesReturned = reqStruct.select(sum("bytes").as("bytes")).collect.head.getAs[Double]("bytes").toLong

totalBytesReturned: Long = 53656682848L

In [12]:
println(s"Número de hosts únicos. $uniqueHostsCount")
println(s"O total de erros 404. $codes404Count")
println(s"Os 5 URLs que mais causaram erro 404. ${top5Hosts404.mkString(",")}")
println(s"Quantidade de erros 404 por dia. ${amount404PerDay.mkString(",")}")
println(s"O total de bytes retornados. $totalBytesReturned")

Número de hosts únicos. 75060
O total de erros 404. 20112
Os 5 URLs que mais causaram erro 404. dialip-217.den.mmc.com,piweba3y.prodigy.com,155.148.25.4,maz3.maz.net,gate.barr.com
Quantidade de erros 404 por dia. (1995-08-08,792),(1995-08-31,1102),(1995-08-01,486),(1995-08-18,514),(1995-08-21,620),(1995-08-16,506),(1995-08-11,544),(1995-08-17,552),(1995-08-26,716),(1995-08-24,832),(1995-08-07,1048),(1995-08-23,718),(1995-08-22,532),(1995-08-25,838),(1995-08-09,568),(1995-08-04,698),(1995-08-30,1078),(1995-08-20,612),(1995-08-13,440),(1995-08-29,832),(1995-08-06,758),(1995-08-15,648),(1995-08-03,576),(1995-08-10,624),(1995-08-14,594),(1995-08-19,426),(1995-09-01,52),(1995-08-05,474),(1995-08-27,768),(1995-08-28,812),(1995-08-12,352)
O total de bytes retornados. 53656682848
